In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
tf.keras.backend.clear_session()

In [6]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [7]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [8]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Minas Gerais - Produção de Cimento (t)', axis=1)
data

,Ano,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),...,Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Minas Gerais - Consumo de Cimento (t)
0,2008,0.738122,0.717622,3.117564e+08,1.831109e+07,14.350809,2.815599e+08,8.001633,0.765604,5.483335e+08,...,11.760983,0.846543,20.050304,888.816223,1175.078681,18.584944,14.242417,78.155036,2.404497,5902.354000
1,2009,0.737078,0.721167,3.136613e+08,1.851096e+07,14.359230,2.816401e+08,8.031078,0.767083,5.302764e+08,...,9.472585,0.848819,20.118491,888.717863,1187.249071,18.626049,14.472669,79.056377,2.439866,6032.354000
2,2010,0.736297,0.724635,3.216212e+08,1.853410e+07,14.420260,2.834821e+08,7.994906,0.768464,5.046425e+08,...,9.332485,0.850522,20.152718,888.783005,1200.816603,18.504253,14.480539,79.770914,2.516811,6751.429000
3,2011,0.735777,0.728021,3.283955e+08,1.849638e+07,14.438708,2.844049e+08,7.984812,0.769750,4.854622e+08,...,10.985689,0.851665,19.732386,888.946710,1196.199756,18.639750,14.686379,80.239851,2.517000,7134.988000
4,2012,0.735118,0.730837,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.770583,4.867661e+08,...,8.159014,0.852207,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120,7144.026000
5,2013,0.735878,0.734823,3.388207e+08,1.820659e+07,14.387080,2.847505e+08,9.529468,0.772298,5.225848e+08,...,7.913561,0.852224,18.647185,889.059458,1261.577485,18.602421,14.037685,77.312871,2.248054,7225.971000
6,2014,0.736117,0.738392,3.422472e+08,1.797983e+07,14.350903,2.842942e+08,9.983692,0.773490,5.676413e+08,...,10.350442,0.851826,19.420470,889.109318,1283.285314,18.788690,13.989516,76.228776,2.141659,6631.735714
7,2015,0.735877,0.740091,3.447299e+08,1.774932e+07,14.320707,2.838475e+08,10.328071,0.773781,5.179392e+08,...,12.454398,0.850979,19.834213,889.120461,1287.845984,18.774990,14.000459,76.241248,2.086753,5836.891371
8,2016,0.730090,0.749468,3.468708e+08,1.763097e+07,14.321241,2.839239e+08,10.248043,0.778559,4.524144e+08,...,13.111908,0.862294,19.849624,889.101242,1284.678488,18.570380,13.855280,76.453950,2.040880,5091.098914
9,2017,0.730553,0.750419,3.480263e+08,1.759670e+07,14.339149,2.842415e+08,9.789036,0.778792,4.213689e+08,...,9.479812,0.861503,20.552463,889.087988,1271.437296,18.605289,13.974131,76.845115,1.808195,4898.010000


In [9]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.738122,0.717622,3.117564e+08,1.831109e+07,14.350809,2.815599e+08,8.001633,0.765604,5.483335e+08,2.546644e+09,...,2.868108,11.760983,0.846543,20.050304,888.816223,1175.078681,18.584944,14.242417,78.155036,2.404497
1,0.737078,0.721167,3.136613e+08,1.851096e+07,14.359230,2.816401e+08,8.031078,0.767083,5.302764e+08,2.679945e+09,...,2.305885,9.472585,0.848819,20.118491,888.717863,1187.249071,18.626049,14.472669,79.056377,2.439866
2,0.736297,0.724635,3.216212e+08,1.853410e+07,14.420260,2.834821e+08,7.994906,0.768464,5.046425e+08,2.891930e+09,...,1.958246,9.332485,0.850522,20.152718,888.783005,1200.816603,18.504253,14.480539,79.770914,2.516811
3,0.735777,0.728021,3.283955e+08,1.849638e+07,14.438708,2.844049e+08,7.984812,0.769750,4.854622e+08,3.091988e+09,...,2.079088,10.985689,0.851665,19.732386,888.946710,1196.199756,18.639750,14.686379,80.239851,2.517000
4,0.735118,0.730837,3.341102e+08,1.839259e+07,14.419985,2.847525e+08,9.070299,0.770583,4.867661e+08,3.189506e+09,...,2.183282,8.159014,0.852207,19.123801,888.986931,1216.021140,18.702693,14.531110,79.216479,2.398120
5,0.735878,0.734823,3.388207e+08,1.820659e+07,14.387080,2.847505e+08,9.529468,0.772298,5.225848e+08,3.093872e+09,...,2.403445,7.913561,0.852224,18.647185,889.059458,1261.577485,18.602421,14.037685,77.312871,2.248054
6,0.736117,0.738392,3.422472e+08,1.797983e+07,14.350903,2.842942e+08,9.983692,0.773490,5.676413e+08,2.779340e+09,...,2.748397,10.350442,0.851826,19.420470,889.109318,1283.285314,18.788690,13.989516,76.228776,2.141659
7,0.735877,0.740091,3.447299e+08,1.774932e+07,14.320707,2.838475e+08,10.328071,0.773781,5.179392e+08,2.561731e+09,...,2.501529,12.454398,0.850979,19.834213,889.120461,1287.845984,18.774990,14.000459,76.241248,2.086753
8,0.730090,0.749468,3.468708e+08,1.763097e+07,14.321241,2.839239e+08,10.248043,0.778559,4.524144e+08,2.516859e+09,...,2.073464,13.111908,0.862294,19.849624,889.101242,1284.678488,18.570380,13.855280,76.453950,2.040880
9,0.730553,0.750419,3.480263e+08,1.759670e+07,14.339149,2.842415e+08,9.789036,0.778792,4.213689e+08,2.497710e+09,...,2.177386,9.479812,0.861503,20.552463,889.087988,1271.437296,18.605289,13.974131,76.845115,1.808195


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     6032.354000
1     6751.429000
2     7134.988000
3     7144.026000
4     7225.971000
5     6631.735714
6     5836.891371
7     5091.098914
8     4898.010000
9     5509.999000
10    6256.450000
11    6954.530000
12    6975.675000
13    6706.893000
14            NaN
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [11]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.323232,-1.620866,-1.838149,0.673524,-0.408054,-2.145004,-1.321648,-1.614153,1.204752,-0.990234,...,2.026352,0.984445,-1.468389,0.546435,-1.229262,-1.471435,-0.707157,0.340929,0.384164,0.760640
1,0.960748,-1.314156,-1.689631,1.223073,-0.189336,-2.066959,-1.288472,-1.285800,0.820657,-0.546688,...,0.085558,-0.051710,-1.017356,0.681014,-1.970482,-1.167751,-0.184239,0.652313,0.732526,0.855996
2,0.689922,-1.014059,-1.069034,1.286693,1.395717,-0.274561,-1.329227,-0.979439,0.275392,0.158672,...,-1.114490,-0.115145,-0.679745,0.748566,-1.479584,-0.829204,-1.733683,0.662956,1.008690,1.063440
3,0.509488,-0.721005,-0.540878,1.182986,1.874830,0.623361,-1.340601,-0.693916,-0.132596,0.824346,...,-0.697344,0.633403,-0.453278,-0.081023,-0.245950,-0.944407,-0.009940,0.941327,1.189931,1.063951
4,0.280737,-0.477317,-0.095331,0.897610,1.388579,0.961592,-0.117537,-0.509072,-0.104860,1.148829,...,-0.337667,-0.646476,-0.345909,-1.282156,0.057146,-0.449809,0.790783,0.731347,0.794405,0.743448
5,0.544382,-0.132344,0.271919,0.386180,0.533962,0.959671,0.399829,-0.128412,0.657046,0.830615,...,0.422337,-0.757614,-0.342490,-2.222830,0.603696,0.686946,-0.484821,0.064056,0.058673,0.338865
6,0.627589,0.176473,0.539068,-0.237305,-0.405597,0.515649,0.911621,0.136019,1.615453,-0.215960,...,1.613108,0.345772,-0.421353,-0.696635,0.979423,1.228615,1.884804,-0.001086,-0.360323,0.052022
7,0.544238,0.323529,0.732633,-0.871091,-1.189838,0.081036,1.299648,0.200563,0.558230,-0.940033,...,0.760923,1.298413,-0.589358,0.119948,1.063393,1.342416,1.710518,0.013713,-0.355502,-0.096007
8,-1.463830,1.134917,0.899544,-1.196495,-1.175976,0.155328,1.209478,1.261018,-0.835563,-1.089341,...,-0.716758,1.596124,1.653000,0.150363,0.918566,1.263379,-0.892435,-0.182621,-0.273294,-0.219681
9,-1.303270,1.217266,0.989637,-1.290727,-0.710864,0.464412,0.692295,1.312653,-1.495939,-1.153056,...,-0.358019,-0.048438,1.496166,1.537521,0.818685,0.932975,-0.448342,-0.021892,-0.122112,-0.847007


In [12]:
reshaped_train_input = dfToInputRNN(train_input)

In [13]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[-1.22797939,  1.13807194,  0.80796688, -0.88251021,
         -0.58408809,  0.25850469,  0.40679149,  1.04994977,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042,  0

In [14]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     6032.354000
1     6751.429000
2     7134.988000
3     7144.026000
4     7225.971000
5     6631.735714
6     5836.891371
7     5091.098914
8     4898.010000
9     5509.999000
10    6256.450000
11    6954.530000
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [15]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    6954.53
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [16]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.340422,0.984852,0.49047,-0.775222,-1.103617,-0.390774,0.251054,0.738141,1.108405,2.635823,...,-1.482779,-2.260447,0.453072,-0.236248,-1.371702,-1.653764,-1.48158,-3.104735,-2.983429,-1.314068


In [17]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-1.34042219,  0.98485229,  0.49047019, -0.77522176,
         -1.10361718, -0.39077384,  0.25105411,  0.73814139,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738,  0

In [18]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    6975.675
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [20]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [21]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [23]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3100286280, 3964565783, 2936922380, 1361037154, 2471886551, 998412215, 1694806791, 1594869765, 3278811470, 1573077532, 4084719352, 4132059979, 1656956435, 1570450596, 817443742, 2610700755, 3175144843, 3199251040, 1711182900, 1725551758, 1373773124, 3553020638, 1489078994, 3868528801, 565733916, 1728650872, 3312587755, 3895091539, 1702594133, 3206432509, 2110887261, 955702360, 3086874781, 1206500496, 1213192623, 3431392176, 3177738997, 3750503525, 1184274513, 1780314845, 421483774, 1689777890, 1561810936, 1000024559, 2186674202, 1194516429, 2985303742, 2932590420, 2079837348, 145994523, 2124497702, 1212795543, 2183157326, 2722218240, 3280883038, 1385539149, 458205271, 3127881546, 2573621066, 3080582975, 1471936174, 3314947169, 2128571345, 3012659509, 1361158785, 1001412626, 2542871884, 401076440, 4278998280, 2462094291, 37807421, 1975972737, 2722455497, 214095724, 3653360541, 518118895, 3847036733, 4020194128, 3410248437, 2527225558, 665506177, 1523420986, 3343406948, 3306831271, 1809

loss: 26315.466796875


Step: 93 ___________________________________________
loss: 44464.44921875


Step: 94 ___________________________________________
loss: 66830.9453125


Step: 95 ___________________________________________
loss: 41943.27734375


Step: 96 ___________________________________________
loss: 49912.81640625


Step: 97 ___________________________________________
loss: 44545.80859375


Step: 98 ___________________________________________
loss: 45767.26953125


Step: 99 ___________________________________________
loss: 54168.375


final_seed: 3199251040


In [24]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 40163236.0000
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 39302192.0000
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 25611178.0000
Epoch 4/10000
1/1 [==============================] - 0s 7ms/step - loss: 10538521.0000
Epoch 5/10000
1/1 [==============================] - 0s 9ms/step - loss: 844403.0625
Epoch 6/10000
1/1 [==============================] - 0s 9ms/step - loss: 656301.9375
Epoch 7/10000
1/1 [==============================] - 0s 7ms/step - loss: 403619.0312
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 714967.2500
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 831310.0000
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 669107.6875
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 910093.9375
Epoch 12/10000
1/1 [==============================] - 0s

1/1 [==============================] - 0s 5ms/step - loss: 385824.5312
Epoch 97/10000
1/1 [==============================] - 0s 4ms/step - loss: 239525.4531
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 365345.0938
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 229638.1719
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 295501.2188
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 309010.3125
Epoch 102/10000
1/1 [==============================] - 0s 5ms/step - loss: 112014.1250
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 391109.5938
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 471600.3438
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 190429.5625
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 322604.7188
Epoch 107/10000
1/1 [==============================] - 0s 4ms/

1/1 [==============================] - 0s 5ms/step - loss: 166928.2031
Epoch 191/10000
1/1 [==============================] - 0s 6ms/step - loss: 340204.0938
Epoch 192/10000
1/1 [==============================] - 0s 6ms/step - loss: 336143.4062
Epoch 193/10000
1/1 [==============================] - 0s 4ms/step - loss: 258324.7500
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 275979.0000
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 275894.3438
Epoch 196/10000
1/1 [==============================] - 0s 6ms/step - loss: 513702.0000
Epoch 197/10000
1/1 [==============================] - 0s 7ms/step - loss: 260277.8281
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 277353.1250
Epoch 199/10000
1/1 [==============================] - 0s 6ms/step - loss: 362204.4062
Epoch 200/10000
1/1 [==============================] - 0s 6ms/step - loss: 420359.5938
Epoch 201/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 5ms/step - loss: 151899.0156
Epoch 285/10000
1/1 [==============================] - 0s 5ms/step - loss: 132956.4375
Epoch 286/10000
1/1 [==============================] - 0s 4ms/step - loss: 412852.8438
Epoch 287/10000
1/1 [==============================] - 0s 4ms/step - loss: 259549.1719
Epoch 288/10000
1/1 [==============================] - 0s 5ms/step - loss: 310156.8438
Epoch 289/10000
1/1 [==============================] - 0s 5ms/step - loss: 274969.3125
Epoch 290/10000
1/1 [==============================] - 0s 5ms/step - loss: 351409.1562
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 451438.8438
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 203784.6875
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 294123.3750
Epoch 294/10000
1/1 [==============================] - 0s 5ms/step - loss: 189783.8750
Epoch 295/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 5ms/step - loss: 220092.3906
Epoch 379/10000
1/1 [==============================] - 0s 5ms/step - loss: 236459.7656
Epoch 380/10000
1/1 [==============================] - 0s 5ms/step - loss: 123076.4922
Epoch 381/10000
1/1 [==============================] - 0s 6ms/step - loss: 230623.1250
Epoch 382/10000
1/1 [==============================] - 0s 6ms/step - loss: 215725.0781
Epoch 383/10000
1/1 [==============================] - 0s 7ms/step - loss: 326207.2812
Epoch 384/10000
1/1 [==============================] - 0s 6ms/step - loss: 339945.8438
Epoch 385/10000
1/1 [==============================] - 0s 7ms/step - loss: 196123.8750
Epoch 386/10000
1/1 [==============================] - 0s 6ms/step - loss: 306187.5938
Epoch 387/10000
1/1 [==============================] - 0s 6ms/step - loss: 252262.2500
Epoch 388/10000
1/1 [==============================] - 0s 5ms/step - loss: 199766.3125
Epoch 389/10000
1/1 [==============================] - 0s 5

...

1/1 [==============================] - 0s 4ms/step - loss: 244188.3594
Epoch 1967/10000
1/1 [==============================] - 0s 4ms/step - loss: 294017.9688
Epoch 1968/10000
1/1 [==============================] - 0s 5ms/step - loss: 127423.6641
Epoch 1969/10000
1/1 [==============================] - 0s 5ms/step - loss: 321262.3438
Epoch 1970/10000
1/1 [==============================] - 0s 6ms/step - loss: 263104.2500
Epoch 1971/10000
1/1 [==============================] - 0s 6ms/step - loss: 227098.2500
Epoch 1972/10000
1/1 [==============================] - 0s 5ms/step - loss: 337828.7812
Epoch 1973/10000
1/1 [==============================] - 0s 5ms/step - loss: 142114.6719
Epoch 1974/10000
1/1 [==============================] - 0s 4ms/step - loss: 325693.0312
Epoch 1975/10000
1/1 [==============================] - 0s 4ms/step - loss: 226888.3125
Epoch 1976/10000
1/1 [==============================] - 0s 5ms/step - loss: 151777.5156
Epoch 1977/10000
1/1 [===========================

1/1 [==============================] - 0s 5ms/step - loss: 161523.4375
Epoch 2060/10000
1/1 [==============================] - 0s 6ms/step - loss: 260252.7500
Epoch 2061/10000
1/1 [==============================] - 0s 5ms/step - loss: 153637.1875
Epoch 2062/10000
1/1 [==============================] - 0s 4ms/step - loss: 359938.5312
Epoch 2063/10000
1/1 [==============================] - 0s 4ms/step - loss: 237438.7031
Epoch 2064/10000
1/1 [==============================] - 0s 5ms/step - loss: 364782.5312
Epoch 2065/10000
1/1 [==============================] - 0s 4ms/step - loss: 267406.9688
Epoch 2066/10000
1/1 [==============================] - 0s 5ms/step - loss: 256751.8594
Epoch 2067/10000
1/1 [==============================] - 0s 5ms/step - loss: 202924.4219
Epoch 2068/10000
1/1 [==============================] - 0s 4ms/step - loss: 260058.9219
Epoch 2069/10000
1/1 [==============================] - 0s 5ms/step - loss: 156576.0625
Epoch 2070/10000
1/1 [===========================

1/1 [==============================] - 0s 4ms/step - loss: 181409.5000
Epoch 2153/10000
1/1 [==============================] - 0s 4ms/step - loss: 259613.8281
Epoch 2154/10000
1/1 [==============================] - 0s 4ms/step - loss: 135709.8750
Epoch 2155/10000
1/1 [==============================] - 0s 5ms/step - loss: 176683.1250
Epoch 2156/10000
1/1 [==============================] - 0s 4ms/step - loss: 192061.7500
Epoch 2157/10000
1/1 [==============================] - 0s 4ms/step - loss: 181759.0156
Epoch 2158/10000
1/1 [==============================] - 0s 4ms/step - loss: 466818.9688
Epoch 2159/10000
1/1 [==============================] - 0s 6ms/step - loss: 277597.4062
Epoch 2160/10000
1/1 [==============================] - 0s 5ms/step - loss: 306513.6562
Epoch 2161/10000
1/1 [==============================] - 0s 5ms/step - loss: 240294.3906
Epoch 2162/10000
1/1 [==============================] - 0s 5ms/step - loss: 166775.5156
Epoch 2163/10000
1/1 [===========================

1/1 [==============================] - 0s 5ms/step - loss: 225722.5781
Epoch 2246/10000
1/1 [==============================] - 0s 4ms/step - loss: 259376.2031
Epoch 2247/10000
1/1 [==============================] - 0s 4ms/step - loss: 140431.7188
Epoch 2248/10000
1/1 [==============================] - 0s 4ms/step - loss: 301111.2188
Epoch 2249/10000
1/1 [==============================] - 0s 4ms/step - loss: 360292.5938
Epoch 2250/10000
1/1 [==============================] - 0s 4ms/step - loss: 94020.4453
Epoch 2251/10000
1/1 [==============================] - 0s 5ms/step - loss: 216286.7969
Epoch 2252/10000
1/1 [==============================] - 0s 5ms/step - loss: 288182.0000
Epoch 2253/10000
1/1 [==============================] - 0s 5ms/step - loss: 295384.9062
Epoch 2254/10000
1/1 [==============================] - 0s 6ms/step - loss: 244335.6250
Epoch 2255/10000
1/1 [==============================] - 0s 6ms/step - loss: 368072.3438
Epoch 2256/10000
1/1 [============================

1/1 [==============================] - 0s 5ms/step - loss: 167293.4219
Epoch 2339/10000
1/1 [==============================] - 0s 5ms/step - loss: 259465.6250
Epoch 2340/10000
1/1 [==============================] - 0s 4ms/step - loss: 72028.8828
Epoch 2341/10000
1/1 [==============================] - 0s 4ms/step - loss: 365108.4688
Epoch 2342/10000
1/1 [==============================] - 0s 4ms/step - loss: 113364.8516
Epoch 2343/10000
1/1 [==============================] - 0s 4ms/step - loss: 329713.0312
Epoch 2344/10000
1/1 [==============================] - 0s 4ms/step - loss: 126612.2109
Epoch 2345/10000
1/1 [==============================] - 0s 5ms/step - loss: 226616.0625
Epoch 2346/10000
1/1 [==============================] - 0s 4ms/step - loss: 188324.8594
Epoch 2347/10000
1/1 [==============================] - 0s 4ms/step - loss: 275114.1562
Epoch 2348/10000
1/1 [==============================] - 0s 4ms/step - loss: 275645.8438
Epoch 2349/10000
1/1 [============================

In [25]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 586ms/step
[6975.675](test_target) - [[6425.662]](prediction) = 550.0128906250002


In [26]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.07884726433284237

In [27]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [28]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.279372,-1.229169,-0.918215,-1.407905,-0.838794,-0.751794,-0.482076,-1.238643,1.148205,-1.122043,...,1.308949,1.412346,-1.279057,-1.335859,1.073554,-1.201994,0.260829,-1.413615,-1.269117,-1.049915
1,-0.117760,0.008898,-0.472376,0.588399,-0.566665,-0.661457,1.392429,0.028285,0.140893,-0.184474,...,-0.190804,-0.643250,0.117025,0.265921,-1.334029,-0.044300,1.073320,0.671180,0.094182,-0.295566
2,-1.161611,1.220272,1.390591,0.819506,1.405459,1.413251,-0.910354,1.210357,-1.289098,1.306518,...,-1.118144,-0.769096,1.162032,1.069938,0.260475,1.246294,-1.334149,0.742435,1.174935,1.345481


train_target:


,Minas Gerais - Consumo de Cimento (t)
0,6032.354
1,6751.429
2,7134.988


test_input:


...

,Minas Gerais - IDH Renda,Minas Gerais - IDH Educacao,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Desemprego,Minas Gerais - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Minas Gerais - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-1.340422,0.984852,0.49047,-0.775222,-1.103617,-0.390774,0.251054,0.738141,1.108405,2.635823,...,-1.482779,-2.260447,0.453072,-0.236248,-1.371702,-1.653764,-1.48158,-3.104735,-2.983429,-1.314068


test_target:


,Minas Gerais - Consumo de Cimento (t)
12,6975.675


1/1 [==============================] - 1s 556ms/step
Error: 550.0128906250002




[7134.962890625,
 7161.94921875,
 6671.9287109375,
 6295.46435546875,
 5560.986328125,
 5319.439453125,
 4932.84033203125,
 5112.83642578125,
 5868.8759765625,
 6425.662109375]

In [29]:
display(targets)
display(predictions)

[7144.026,
 7225.971,
 6631.735714285716,
 5836.891371428574,
 5091.098914285716,
 4898.01,
 5509.999,
 6256.45,
 6954.53,
 6975.675]

[7134.962890625,
 7161.94921875,
 6671.9287109375,
 6295.46435546875,
 5560.986328125,
 5319.439453125,
 4932.84033203125,
 5112.83642578125,
 5868.8759765625,
 6425.662109375]

In [30]:
mae = mean_absolute_error(predictions, targets)
mae

481.96068945312425

In [31]:
porcentage = mae/np.mean(targets)
porcentage

0.07708363289561307